In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import PCA
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared
from sklearn.model_selection import GridSearchCV
from supporting import remove_outliers_y, assign_val_data_to_clusters_with_labels,assign_test_data_to_clusters_with_labels, tune_ridge_regression, plot_clusters_features_vs_target, divide_data_by_kmeans_distance, remove_outliers, standardize_data, create_polynomial_features, apply_pca, rmse, tune_kernel_ridge_regression, tune_gaussian_process_regression

In [32]:
# importing the data
training_data = np.genfromtxt("training_data.csv", delimiter=',')
X = training_data[:,:-1]
y = training_data[:,-1]

X, y = remove_outliers_y(X, y, 50)

1


In [41]:
#####################################################
# USE THIS CELL TO TRAIN AND CREATE YOUR BEST MODEL # 
#####################################################

# Data Preprocessing

# based on our previous analysis and the random_state we set for KMeans clustering we know that:
# cluster 0 should be solved using Averaging (mean of features 2 > 61)
# cluster 1 should be solved using Bayesian Ridge (mean of features 2 <= 61)

def BayesianTrain(X_train, y_train):
    model = BayesianRidge()
    return model.fit(X_train, y_train)

def AveragingTrain(X_train, y_train):
    # Create the KNN Regressor model
    model_knr =  KNeighborsRegressor(n_neighbors=10, metric = 'manhattan')
    model_g = GaussianProcessRegressor(alpha=0.1, kernel=RationalQuadratic(alpha=0.1, length_scale=50), normalize_y= True)

    scaler = StandardScaler()

    # Fit the scaler on the training data and transform both the training and testing data
    X_train_knr = scaler.fit_transform(X_train)

    pca_transformer = PCA(n_components=0.97)
    # Fit the PCA transformer on the training data and transform both the training and testing data
    X_train_knr = pca_transformer.fit_transform(X_train_knr)

    poly_transformer = PolynomialFeatures(degree=3, include_bias=False)

    # Fit the transformer on the training data and transform both the training and testing data
    X_train_g = poly_transformer.fit_transform(X_train)

    # Fit the model to the training data
    return model_knr.fit(X_train_knr, y_train), model_g.fit(X_train_g, y_train), scaler, pca_transformer, poly_transformer

def training_model(X, y, random_state=40):
    # Divide the data using KMeans
    (X_train_clusters, y_train_clusters), clusters, labels = divide_data_by_kmeans_distance(
        X, y, n_clusters=2, random_state=random_state
    )

    # Calculate the mean of the second feature for each cluster
    cluster_0_mean = clusters[0][1]
    cluster_1_mean = clusters[1][1]

    # This condition was only inserted to make sure that there was no issue in labels' assignment
    # Ensure that cluster 1 has a mean of the second feature below 61 and cluster 2 above 61
    if cluster_0_mean < 61 and cluster_1_mean > 61:
        print('Correct')
        X1_train, y1_train = X_train_clusters[0], y_train_clusters[0]
        X2_train, y2_train = X_train_clusters[1], y_train_clusters[1]
    elif cluster_1_mean < 61 and cluster_0_mean > 61:
        print('Had to switch')
        X1_train, y1_train = X_train_clusters[1], y_train_clusters[1]
        X2_train, y2_train = X_train_clusters[0], y_train_clusters[0]
        labels = [1, 0]
    else:
        raise ValueError("The clusters do not meet the expected conditions for the second feature.")

    # Train the models
    best_model_knr1, best_model_g1, scaler, pca_transformer, poly_transformer = AveragingTrain(X1_train, y1_train)
    best_model2 = BayesianTrain(X2_train, y2_train)

    def predict(X_test):
        
        # Get the clustered test data and their original indices
        X1_test, X2_test, idx1, idx2 = assign_test_data_to_clusters_with_labels(X_test, clusters, labels)

        X1_test_knr = scaler.transform(X1_test)

        X1_test_knr = pca_transformer.transform(X1_test_knr)

        # Fit the transformer on the training data and transform both the training and testing data
        X1_test_g = poly_transformer.transform(X1_test)

        # Handle case where no data is assigned to one of the clusters
        if X2_test.shape[0] == 0:
            # If no data is assigned to X2_test, treat the entire set as belonging to cluster 1 (KNR and GPR)
            X2_test = X1_test
            idx2 = idx1  # Update indices to ensure consistency
        elif X1_test.shape[0] == 0:
            # If no data is assigned to X1_test, treat the entire set as belonging to cluster 2 (Bayesian Ridge)
            X1_test = X2_test
            idx1 = idx2  # Update indices to ensure consistency

        # Make predictions for each cluster
        y2_pred = best_model2.predict(X2_test)  # Predictions for Bayesian Ridge
        y1_pred_g = best_model_g1.predict(X1_test_g)  # Predictions for Gaussian Process
        y1_pred_knr = best_model_knr1.predict(X1_test_knr)  # Predictions for KNN Regressor

        # Calculate inverse variances and handle case where variance is zero
        inv_var_knr = 1 / np.var(y1_pred_knr) if np.var(y1_pred_knr) != 0 else 0
        inv_var_g = 1 / np.var(y1_pred_g) if np.var(y1_pred_g) != 0 else 0

        total_variance = inv_var_knr + inv_var_g if inv_var_knr + inv_var_g != 0 else 1  # Avoid division by zero

        w_knr = inv_var_knr / total_variance
        w_g = inv_var_g / total_variance

        # Initialize a full array for y predictions
        y_pred_full = np.empty(X_test.shape[0])

        # Place predictions back in their original order
        if X1_test.shape[0] > 0:
            y_pred_full[idx1] = y1_pred_knr * w_knr + y1_pred_g * w_g
        if X2_test.shape[0] > 0:
            y_pred_full[idx2] = y2_pred

        return y_pred_full, y1_pred_knr, y2_pred, idx1, idx2


    return predict

In [42]:
# Loop to evaluate RMSE over multiple splits

from sklearn.metrics import r2_score

sum_test = 0
sum_train = 0

sum_r2 = 0
sum_r2_train = 0

for i in range(0, 41):
    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i)
    filtered_indices = y_test < 61  # Create a boolean mask for the condition

    # Apply the mask to both x_test and y_test
    x_test_filtered = x_test[filtered_indices]  # Filter x_test
    y_test_filtered = y_test[filtered_indices]

    # Train and predict
    best_model = training_model(x_train, y_train)
    y_pred_full, y1_pred_knr, y2_pred, idx1, idx2 = best_model(x_test)

    y_pred_train, y1_pred_train, y2_pred_train, idx1_train, idx2_train = best_model(x_train)

    # Calculate RMSE for full test set, cluster 1, and cluster 2
    rmse_full = rmse(y_test, y_pred_full)
    rmse_cluster1 = rmse(y_test[idx1], y1_pred_knr)
    rmse_cluster2 = rmse(y_test[idx2], y2_pred)

    rmse_full_train = rmse(y_train, y_pred_train)
    rmse_cluster1_train = rmse(y_train[idx1_train], y1_pred_train)
    rmse_cluster2_train = rmse(y_train[idx2_train], y2_pred_train)

    # Print RMSE for each iteration
    print(f"Iteration {i+1} - Full RMSE: {rmse_full}, Cluster 1 RMSE: {rmse_cluster1}, Cluster 2 RMSE: {rmse_cluster2}")
    print(f"Iteration {i+1} - Train RMSE: {rmse_full_train}, Cluster 1 RMSE: {rmse_cluster1_train}, Cluster 2 RMSE: {rmse_cluster2_train}")
    print(r2_score(y_test, y_pred_full))
    print(r2_score(y_train, y_pred_train))

    # Accumulate RMSE for average
    sum_test += rmse_full
    sum_train += rmse_full_train
    sum_r2 += r2_score(y_test, y_pred_full)
    sum_r2_train += r2_score(y_train, y_pred_train)

# Print average RMSE
print(f"Average RMSE over 40 splits: {sum_test / 40}")
print(f"Average RMSE over 40 splits: {sum_train / 40}")
print(f"Average RMSE over 40 splits: {sum_r2 / 40}")
print(f"Average RMSE over 40 splits: {sum_r2_train / 40}")


Correct


/Users/paolocadei/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 1 - Full RMSE: 2.824229643448717, Cluster 1 RMSE: 3.9495945858016763, Cluster 2 RMSE: 7.241879337284717e-10
Iteration 1 - Train RMSE: 2.5825123124811165, Cluster 1 RMSE: 4.096381198291937, Cluster 2 RMSE: 7.516078894413401e-10
0.6829116150518442
0.7413194341251914
Correct


/Users/paolocadei/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 2 - Full RMSE: 1.46135696332039, Cluster 1 RMSE: 2.189875372981512, Cluster 2 RMSE: 1.1373972548065722e-09
Iteration 2 - Train RMSE: 2.665667469981313, Cluster 1 RMSE: 4.2305309156975355, Cluster 2 RMSE: 8.710311690444489e-10
0.8155017102441486
0.765350490992157
Correct


...

Iteration 39 - Full RMSE: 1.958862049635708, Cluster 1 RMSE: 2.9803826667569497, Cluster 2 RMSE: 9.556360358470847e-10
Iteration 39 - Train RMSE: 2.743920474468158, Cluster 1 RMSE: 4.282075126782758, Cluster 2 RMSE: 8.359849617057117e-10
0.7723753139889526
0.7361157996051029
Had to switch


/Users/paolocadei/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 40 - Full RMSE: 2.227943739755706, Cluster 1 RMSE: 3.180628192249839, Cluster 2 RMSE: 8.946283111640911e-10
Iteration 40 - Train RMSE: 2.522337869353307, Cluster 1 RMSE: 4.074982117856816, Cluster 2 RMSE: 8.212321859015394e-10
0.6559075519835764
0.7831210100112265
Correct


/Users/paolocadei/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 41 - Full RMSE: 4.243651743160595, Cluster 1 RMSE: 5.647272784000874, Cluster 2 RMSE: 8.817374044852929e-10
Iteration 41 - Train RMSE: 2.317263775517244, Cluster 1 RMSE: 3.719003536813376, Cluster 2 RMSE: 7.559999331972368e-10
0.5377270840274121
0.7456168288700262
Average RMSE over 40 splits: 3.1156937231174857
Average RMSE over 40 splits: 2.500120428031903
Average RMSE over 40 splits: 0.6289710730191576
Average RMSE over 40 splits: 0.7879426601098933


In [51]:
#####################################################
# USE THIS CELL TO TRAIN AND CREATE YOUR BEST MODEL # 
#####################################################

# Data Preprocessing

# based on our previous analysis and the random_state we set for KMeans clustering we know that:
# cluster 0 should be solved using Averaging (mean of features 2 > 61)
# cluster 1 should be solved using Bayesian Ridge (mean of features 2 <= 61)

def BayesianTrain(X_train, y_train):
    model = BayesianRidge()
    return model.fit(X_train, y_train)

def AveragingTrain(X_train, y_train):
    # Create the KNN Regressor model
    model_knr = KNeighborsRegressor(n_neighbors=10, metric = 'manhattan')
    model_g = GaussianProcessRegressor(alpha=0.1, kernel=RationalQuadratic(alpha=0.1, length_scale=50))

    scaler = StandardScaler()

    # Fit the scaler on the training data and transform both the training and testing data
    X_train = scaler.fit_transform(X_train)

    pca_transformer = PCA(n_components=0.97)
    # Fit the PCA transformer on the training data and transform both the training and testing data
    X_train = pca_transformer.fit_transform(X_train)

    # Fit the model to the training data
    knr = model_knr.fit(X_train, y_train)
    g = model_g.fit(X_train, y_train)

    y_pred_g = g.predict(X_train)  # Predictions for Gaussian Process
    y_pred_knr = knr.predict(X_train)  # Predictions for KNN Regressor

    # Calculate inverse variances and handle case where variance is zero
    inv_var_knr = 1 / np.var(y_pred_knr) if np.var(y_pred_knr) != 0 else 0
    inv_var_g = 1 / np.var(y_pred_g) if np.var(y_pred_g) != 0 else 0

    total_variance = inv_var_knr + inv_var_g if inv_var_knr + inv_var_g != 0 else 1  # Avoid division by zero

    w_knr = inv_var_knr / total_variance
    w_g = inv_var_g / total_variance

    print(w_g, w_knr)

    return knr, g, scaler, pca_transformer, w_knr, w_g

def training_model(X, y, random_state=40):
    # Divide the data using KMeans
    (X_train_clusters, y_train_clusters), clusters, labels = divide_data_by_kmeans_distance(
        X, y, n_clusters=2, random_state=random_state
    )

    # Calculate the mean of the second feature for each cluster
    cluster_0_mean = clusters[0][1]
    cluster_1_mean = clusters[1][1]

    # This condition was only inserted to make sure that there was no issue in labels' assignment
    # Ensure that cluster 1 has a mean of the second feature below 61 and cluster 2 above 61
    if cluster_0_mean < 61 and cluster_1_mean > 61:
        print('Correct')
        X1_train, y1_train = X_train_clusters[0], y_train_clusters[0]
        X2_train, y2_train = X_train_clusters[1], y_train_clusters[1]
    elif cluster_1_mean < 61 and cluster_0_mean > 61:
        print('Had to switch')
        X1_train, y1_train = X_train_clusters[1], y_train_clusters[1]
        X2_train, y2_train = X_train_clusters[0], y_train_clusters[0]
        labels = [1, 0]
    else:
        raise ValueError("The clusters do not meet the expected conditions for the second feature.")

    # Train the models
    best_model_knr1, best_model_g1, scaler, pca_transformer, w_knr, w_g = AveragingTrain(X1_train, y1_train)
    best_model2 = BayesianTrain(X2_train, y2_train)

    def predict(X_test):
        
        # Get the clustered test data and their original indices
        X1_test, X2_test, idx1, idx2 = assign_test_data_to_clusters_with_labels(X_test, clusters, labels)

        X1_test = scaler.transform(X1_test)

        X1_test = pca_transformer.transform(X1_test)

        # Handle case where no data is assigned to one of the clusters
        if X2_test.shape[0] == 0:
            # If no data is assigned to X2_test, treat the entire set as belonging to cluster 1 (KNR and GPR)
            X2_test = X1_test
            idx2 = idx1  # Update indices to ensure consistency
        elif X1_test.shape[0] == 0:
            # If no data is assigned to X1_test, treat the entire set as belonging to cluster 2 (Bayesian Ridge)
            X1_test = X2_test
            idx1 = idx2  # Update indices to ensure consistency

        # Make predictions for each cluster
        y2_pred = best_model2.predict(X2_test)  # Predictions for Bayesian Ridge
        y1_pred_g = best_model_g1.predict(X1_test)  # Predictions for Gaussian Process
        y1_pred_knr = best_model_knr1.predict(X1_test)  # Predictions for KNN Regressor

        # Initialize a full array for y predictions
        y_pred_full = np.empty(X_test.shape[0])

        # Place predictions back in their original order
        if X1_test.shape[0] > 0:
            y_pred_full[idx1] = y1_pred_knr * w_knr + y1_pred_g * w_g
        if X2_test.shape[0] > 0:
            y_pred_full[idx2] = y2_pred

        return y_pred_full, y1_pred_knr, y2_pred, idx1, idx2


    return predict

In [52]:
# Loop to evaluate RMSE over multiple splits

from sklearn.metrics import r2_score

sum_test = 0
sum_train = 0

sum_r2 = 0
sum_r2_train = 0

for i in range(0, 41):
    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=i)
    filtered_indices = x_test[:, 1] < 61  # Create a boolean mask for the condition

    # Apply the mask to both x_test and y_test
    x_test_filtered = x_test[filtered_indices]  # Filter x_test
    y_test_filtered = y_test[filtered_indices]

    # Train and predict
    best_model = training_model(x_train, y_train)
    y_pred_full, y1_pred_knr, y2_pred, idx1, idx2 = best_model(x_test)

    y_pred_train, y1_pred_train, y2_pred_train, idx1_train, idx2_train = best_model(x_train)

    # Calculate RMSE for full test set, cluster 1, and cluster 2
    rmse_full = rmse(y_test, y_pred_full)
    rmse_cluster1 = rmse(y_test[idx1], y1_pred_knr)
    rmse_cluster2 = rmse(y_test[idx2], y2_pred)

    rmse_full_train = rmse(y_train, y_pred_train)
    rmse_cluster1_train = rmse(y_train[idx1_train], y1_pred_train)
    rmse_cluster2_train = rmse(y_train[idx2_train], y2_pred_train)

    # Print RMSE for each iteration
    print(f"Iteration {i+1} - Full RMSE: {rmse_full}, Cluster 1 RMSE: {rmse_cluster1}, Cluster 2 RMSE: {rmse_cluster2}")
    print(f"Iteration {i+1} - Train RMSE: {rmse_full_train}, Cluster 1 RMSE: {rmse_cluster1_train}, Cluster 2 RMSE: {rmse_cluster2_train}")
    print(r2_score(y_test, y_pred_full))
    print(r2_score(y_train, y_pred_train))

    # Accumulate RMSE for average
    sum_test += rmse_full
    sum_train += rmse_full_train
    sum_r2 += r2_score(y_test, y_pred_full)
    sum_r2_train += r2_score(y_train, y_pred_train)

# Print average RMSE
print(f"Average RMSE over 40 splits: {sum_test / 40}")
print(f"Average RMSE over 40 splits: {sum_train / 40}")
print(f"Average RMSE over 40 splits: {sum_r2 / 40}")
print(f"Average RMSE over 40 splits: {sum_r2_train / 40}")


Correct
0.19630388555068415 0.8036961144493158
Iteration 1 - Full RMSE: 2.773535884238292, Cluster 1 RMSE: 3.9495945858016763, Cluster 2 RMSE: 7.241879337284717e-10
Iteration 1 - Train RMSE: 2.5792429151853296, Cluster 1 RMSE: 4.096381198291937, Cluster 2 RMSE: 7.516078894413401e-10
0.694192664963088
0.7419739860528025
Correct
0.20891410476674285 0.7910858952332572
Iteration 2 - Full RMSE: 1.527191153238027, Cluster 1 RMSE: 2.189875372981512, Cluster 2 RMSE: 1.1373972548065722e-09
Iteration 2 - Train RMSE: 2.658090893061991, Cluster 1 RMSE: 4.2305309156975355, Cluster 2 RMSE: 8.710311690444489e-10
0.7985039591019139
0.7666824752049383
Correct
0.22145006491205954 0.7785499350879405
Iteration 3 - Full RMSE: 3.0814595786368653, Cluster 1 RMSE: 3.9958735630744693, Cluster 2 RMSE: 1.063500167242289e-09
Iteration 3 - Train RMSE: 2.372705055909891, Cluster 1 RMSE: 3.905139559132285, Cluster 2 RMSE: 7.868663293761112e-10
0.5373663115790639
0.7940104146036231
Had to switch
0.18667180348846402 0